In [0]:
"""
Google Colab: ランタイムはTPU推奨
"""

import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import os 
import zipfile

In [4]:
import tensorflow as tf 
tf.__version__

'2.2.0'

In [0]:
INPUT_DIR = "./drive/My Drive/kaggle/m5-forecasting/datas"

def read_data():
    cal = pd.read_csv(f"{INPUT_DIR}calendar.csv")
    stv = pd.read_csv(f"{INPUT_DIR}sales_train_validation.csv")
    ste = pd.read_csv(f"{INPUT_DIR}sales_train_evaluation.csv")
    ss = pd.read_csv(f"{INPUT_DIR}sample_submission.csv")
    sellp = pd.read_csv(f"{INPUT_DIR}sell_prices.csv")
    
    return cal, stv, ste, ss, sellp

In [0]:
def reduce_mem_usage(df, verbose=True):
    """
    目的：メモリサイズの削減
    df: メモリを削減したい DataFrame (pandas.DataFrame)
    verbose: 実行時に、メモリ削減の情報を出力するかどうかを指定(bool)

    ■ 基本思想
    【前提知識】
    pandas で作成したデータフレームのうち数値データは、特に dtype を指定しない場合
    int64 または float64 でデータを作成するので、
    実際のデータよりもこの型が大きいと余計なメモリサイズを確保してしまう。

    【処理内容】
    (1) 入力された DataFrame の column の型を全てチェック(for loop)
    (2) その型が大きい数値データ(int16~int64, float16~float64)ならば、
        そのデータフレームの最大値・最小値をチェック。
        現在処理中のカラムを、上記の最大値・最小値を表せる必要最低限の型に変換する。
        int と floatに分けて処理。

    ────────────────────────────────────────────────────────────────────────
    【変更履歴】
    2020/06/06:
    ■ 35行目
    ifのネストが深かったので、リファクタ。
    Early Continueを入れたので可読性が向上(したはず)。

    ■ 46行目・71行目(置き換え・追加)
    説明変数(関数?)で置き換え。
    columnのtypeがintであるか否かを判定する関数を噛ませている。
    (返り値はbool値)
    """

    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2

    # main loop    
    for col in df.columns:
        col_type = df[col].dtypes

        if col_type not in numerics: 
            continue # Early continue if column type is not numeric
        
        c_min = df[col].min()
        c_max = df[col].max()

        if IsInt(col_type):
            if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                df[col] = df[col].astype(np.int32)
            elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                df[col] = df[col].astype(np.int64)  
        else:
            if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                df[col] = df[col].astype(np.float16)
            elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2

    if verbose: 
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))

    return df


def IsInt(col_type):
    return str(col_type)[:3] == 'int'

In [0]:
NUM_ITEMS = 30490
DATA_PATH = "./drive/My Drive/kaggle/m5-forecasting/datas/training_datas.zip"

In [0]:
"""
zipからデータ読み出し。
展開しないのでディスク容量も圧迫せず済む
"""
def train_data_from_csv_generator(num=NUM_ITEMS, datapath=DATA_PATH):
    with zipfile.ZipFile(datapath) as myzip:
        filelist = myzip.namelist()

        for i, f_name in enumerate(filelist):

            if i == 0:
                continue

            if i > num:
                break

            df = pd.read_csv(myzip.extract(f_name))
            df = reduce_mem_usage(df, verbose=False)
            df = df.fillna(0)
            array = df.values
            yield array

In [0]:
"""
ジェネレータを使わないときはこれを用いる。
※ 全部使うとメモリに乗りきらないので非推奨
"""
def CreateTrainingData(timesteps=28, delay=1, num_samples=30490):
    x_shape = next(train_data_from_csv_generator(num=1)).shape

    train_generator = train_data_from_csv_generator(num=num_samples)

    len_sequence, num_features = x_shape
    sample_batchsize = len_sequence-timesteps+1 - delay

    X_train = np.zeros((sample_batchsize*num_samples, timesteps, num_features))
    Y_train = np.zeros((sample_batchsize*num_samples, timesteps, 1))

    for i, array in enumerate(train_generator):
        for j in range(sample_batchsize - timesteps + 1 -delay):
            X_train[i*sample_batchsize+j, 0: timesteps] = array[j:j+timesteps]
            Y_train[i*sample_batchsize+j, 0: timesteps] = array[j+timesteps:j+2*timesteps , num_features-1].reshape(timesteps, 1)

    return X_train, Y_train

In [0]:
from sklearn import preprocessing, metrics
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM,Dropout
from keras.layers import RepeatVector,TimeDistributed, BatchNormalization
from numpy import array
from keras.models import Sequential, load_model
from keras.optimizers import Adam, RMSprop
#import utils_paths
import re
from tqdm import tqdm
import os

"""
仮のモデル
ハイパーパラメータを引数にとれるよう改造すべき？
※ チューニングができるように
"""
def build_model():
    timesteps = 28
    delay = 1

    n_out_seq_length = 28
    num_y = 1

    train_generator = train_data_from_csv_generator(num=1) 
    x_shape = next(train_generator).shape

    len_sequence, num_features = x_shape

    model = Sequential()

    model.add(LSTM(128, activation='relu', batch_input_shape=(None, timesteps, num_features), return_sequences=False))
    model.add(BatchNormalization())
    model.add(RepeatVector(28))
    model.add(LSTM(32, activation='relu', return_sequences=True))
    model.add(BatchNormalization())
    model.add(Dropout(0.1))  
    model.add(TimeDistributed(Dense(delay, activation="relu")))   # num_y means the shape of y,in some problem(like translate), it can be many.
                                                #In that case, you should set the  activation= 'softmax'
    
    #RMSpropOptimizer = RMSprop(lr=0.001, clipvalue=0.5)
    #model.compile(optimizer=RMSpropOptimizer, loss='mean_squared_error', metrics=["accuracy"])
    model.compile(optimizer="adam", loss='mean_squared_error', metrics=["accuracy"])


    return model

In [27]:
model = build_model()
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               73216     
_________________________________________________________________
batch_normalization_1 (Batch (None, 128)               512       
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 28, 128)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 28, 32)            20608     
_________________________________________________________________
batch_normalization_2 (Batch (None, 28, 32)            128       
_________________________________________________________________
dropout_1 (Dropout)          (None, 28, 32)            0         
_________________________________________________________________
time_distributed_1 (TimeDist (None, 28, 1)            

In [0]:
from keras.utils import Sequence
from keras.models import Sequential

"""
model.fit_generatorを使うためのユーザ定義関数
※ generator を使わないとメモリが死ぬ
"""
class ReccurentTrainGenerator(Sequence):
    def _resetindices(self):
        """
        バッチ生成用のインデックスをランダムに出力
        """
        self.num_called = 0

        all_idx = np.random.permutation(np.arange(self.num_batches))
        remain_idx = np.random.choice(np.arange(self.num_batches),
                                      size=(self.steps_per_epoch*self.batch_size-len(all_idx)),
                                      replace=False)
        
        self.indices = np.hstack([all_idx, remain_idx]).reshape(self.steps_per_epoch, self.batch_size)

    def __init__(self, InputTensor, batch_size, InputSteps=28, OutputSteps=28, delay=1, normalize_factor=None):
        """
        【入力】
        InputTensor: 入力データ(説明変数) データ数("HOBBIES_1_..."などに対応) × データ点数(時系列方向のデータ数) × 特徴量数 のndarray
                     ※ 正解ラベルも、この時系列データからとるのでこれだけ入力すればOK 
        batch_size: バッチサイズ(例えば、timestepが5として、時刻0~4までのデータ、1~5までのデータ、...、10~14までのデータ、
                                をひとまとめにして1データとみなすとする。RNNの場合はこのサイズがバッチサイズに対応する。)
        InputSteps: リカレント層に食わせるデータを、何ステップ前までのデータにするか
        OutputSteps: リカレント層からの出力(予測ステップ数)の設定値
        delay: 目的変数をどの程度遅らせるか？(予測ステップのスタート位置をどの程度後ろにずらすか)
        normalize_factor: 正規化する際のスケーリングをどの程度にするか
        """
        self.train_tensor = InputTensor 

        # 各種パラメータ
        self.num_datas = InputTensor.shape[0]
        self.len_sequence = InputTensor.shape[1]
        self.num_features = InputTensor.shape[2]
        self.batch_size = batch_size
        self.input_steps = InputSteps
        self.output_steps = OutputSteps
        self.delay = delay

        # 現在のエポックでバッチ生成の対象となっているデータ系列
        self.now_data = InputTensor[0]

        # 各データ系列に対し、バッチサイズいくつ作れるか計算するのに必要な値
        self.len_requied_per_batch = InputSteps + (batch_size-1) + (delay-1) + OutputSteps # 訓練データと正解データを作るために必要なサイズ 
        self.num_batches = self.len_sequence - self.len_requied_per_batch + 1              # 作れるバッチの数

        # 1エポック当たりのステップ数
        self.steps_per_epoch = int(np.ceil(self.len_sequence / float(batch_size)))
        
        # バッチ生成用の乱数初期化
        self._resetindices()

        # データ取得用インデックス生成
        self.data_idx = self._reset_dataset_indices(self.num_datas)
        self.num_epoch = 0

        self.normalize_factor = normalize_factor


    def __len__(self):
        """
        1エポックあたりのステップ数をリターン
        """
        return self.steps_per_epoch

    def __getitem__(self, idx):
        """
        データをバッチにまとめて出力
        """
        indices_temp = self.indices[idx] # indices は (steps_per_epoch, batchsize)の array

        batch_x = np.array([self.now_data[i:i+self.input_steps] for i in indices_temp])
        batch_y = np.array([self.now_data[i+self.input_steps+(self.delay-1):i+self.input_steps+(self.delay-1)+self.output_steps, -1] for i in indices_temp]).reshape(self.batch_size, self.output_steps, 1)

        if self.num_called == (self.steps_per_epoch-1):
            self._resetindices()
        else:
            self.num_called += 1

        if self.normalize_factor:
            batch_x = batch_x / self.normalize_factor
            batch_y = batch_y / self.normalize_factor

        return batch_x, batch_y

    def on_epoch_end(self):
        """
        Epoch 終了ごとにデータセットを入れ替える
        (データセット："HOBBIES_1_..."などに対応)
        """
        if self.num_epoch == self.num_datas:
            self.num_epoch = 0
            self.data_idx = self._reset_dataset_indices(self.num_datas)
        else:
            self.num_epoch += 1
        
        next_data_idx = self.data_idx[self.num_epoch]
        self.now_data = self.train_tensor[next_data_idx]


    def _reset_dataset_indices(self, num_datas):
        """
        Epoch毎に入れ替えるデータのインデックスをランダムにするためのメソッド
        """
        return np.random.permutation(np.arange(num_datas))

In [0]:
"""
Generatorに食わせるためのトレーニングデータ(ndarray)作成関数
一度作って np.saveで保存すれば使う必要なし(この関数の実行は時間かかる)
"""
def CreateTrainingTensor(num=30490):
    tg = train_data_from_csv_generator(num=1)
    shape = next(tg).shape
    X_train = np.zeros((num, shape[0], shape[1]))

    train_generator = train_data_from_csv_generator(num=num)

    for i, array in enumerate(train_generator):
        if i/1000 == i//1000:
            print(i)

        X_train[i] = array

    return X_train

In [0]:
# 作成と保存の例
# X_train = CreateTrainingTensor()
# X_train = CreateTrainingTensor()

In [0]:
# 読みだしは基本こっちで
X_train = np.load(INPUT_DIR + "/TrainingTensor.npy")

In [0]:
X_max = np.max(X_train)

RTG = ReccurentTrainGenerator(InputTensor=X_train[:-100], batch_size=128, InputSteps=28, normalize_factor=X_max)
Validation_RTG = ReccurentTrainGenerator(X_train[:-100], batch_size=128, InputSteps=28, normalize_factor=X_max)
# トレーニングデータの一部をバリデーション用にする

In [0]:
from keras.callbacks import EarlyStopping 
 
# Early-stopping: patienceはもう少し大きくとる？
early_stopping = EarlyStopping(patience=5, verbose=1) 

history = model.fit_generator(RTG, epochs=500, verbose=1, validation_data=Validation_RTG, callbacks=[early_stopping])

Epoch 1/500
16/16 [==============================] - 4s 278ms/step - loss: 5.2515e-04 - accuracy: 0.6719 - val_loss: 3.5139e-08 - val_accuracy: 0.6721
Epoch 2/500
16/16 [==============================] - 3s 170ms/step - loss: 1.3748e-06 - accuracy: 0.6720 - val_loss: 3.5619e-08 - val_accuracy: 0.6717
Epoch 3/500
16/16 [==============================] - 3s 171ms/step - loss: 6.1179e-07 - accuracy: 0.6719 - val_loss: 3.5379e-08 - val_accuracy: 0.6718
Epoch 4/500
16/16 [==============================] - 3s 171ms/step - loss: 3.2946e-07 - accuracy: 0.6718 - val_loss: 3.5109e-08 - val_accuracy: 0.6719
Epoch 5/500
16/16 [==============================] - 3s 171ms/step - loss: 1.0600e-07 - accuracy: 0.6721 - val_loss: 3.5860e-08 - val_accuracy: 0.6719
Epoch 6/500
16/16 [==============================] - 3s 181ms/step - loss: 4.2529e-06 - accuracy: 0.6718 - val_loss: 3.5229e-08 - val_accuracy: 0.6719
Epoch 7/500
16/16 [==============================] - 3s 175ms/step - loss: 2.3316e-07 - accura

In [0]:
# 学習結果の保存
model_json_str = model.to_json()
open('LSTM_test_model.json', 'w').write(model_json_str)
model.save_weights('LSTM_test_weights.h5');

In [0]:
"""
提出用データの入力作成関数
(これも結果をnp.saveで保存してしまえば使う必要なし)
"""
def GenerateInputForPrediction(num_samples=30490):
    TIMESTEPS = 28

    train_generator = train_data_from_csv_generator(num=1) 
    x_shape = next(train_generator).shape
    num_features = x_shape[1]

    #X_test = np.zeros((num_samples, TIMESTEPS, num_features))

    train_generator = train_data_from_csv_generator(num=num_samples)

    for i, array in enumerate(train_generator):
        #X_test[i] = array[-TIMESTEPS:]
        yield array[-TIMESTEPS:]

In [0]:
# テストデータ作成と保存

# X_test_generator = GenerateInputForPrediction()
# X_test = np.zeros((30490, 28, 14))
# for i, array in enumerate(X_test_generator):
#     if i / 1000 == i //1000:
#         print(i)
#     X_test[i] = array 
# np.save(INPUT_DIR + "/test_data.npy", X_test)

In [0]:
# テストデータの読み出し
X_test = np.load(INPUT_DIR + "/test_data.npy")

In [0]:
prediction = model.predict(X_test)

In [0]:
"""
予測結果のndarrayを提出形式のcsvに変換する関数
"""

INPUT_DIR = "./drive/My Drive/kaggle/m5-forecasting/datas"

def CreateSubmissionCSV(prediction, save_path=INPUT_DIR):
    # Create Prediction DataFrame
    prediction = np.rint(prediction)
    prediction = prediction.astype("int")
    pred_df = pd.DataFrame(prediction.reshape(30490, 28))

    # Get "id" columns (id の情報だけまとめたファイルを作成した方が軽いはず -> 未実装)
    stv = pd.read_csv(INPUT_DIR + "/sales_train_validation.csv")
    ste = pd.read_csv(INPUT_DIR + "/sales_train_evaluation.csv")

    # Rename Index & Columns
    pred_df.index = list(ste["id"])
    pred_df.columns = [f'F{i}' for i in range(1, 28 + 1)]

    # Set index label "id"
    pred_df = pred_df.reset_index()
    pred_df = pred_df.rename(columns={"index": "id"})
    pred_df = pred_df.set_index("id")

    # Create "Validation" DataFrame
    validation_df = ste[["id"] +  ["d_" + str(i) for i in range(1914, 1942)]]

    # Rename columns & set index "id"
    validation_df = validation_df.set_index("id")
    validation_df = validation_df.rename(columns={"d_" + str(i + 1913): "F" + str(i) for i in range(1, 29)})
    validation_df.index = stv["id"]

    # Create Submission DataFrame (shape = (60980, 28))
    submission_df = pd.concat([validation_df, pred_df], axis=0)
    submission_df.to_csv(save_path + "/submission.csv")

    # For Debug

    print(submission_df.shape)
    return submission_df


In [0]:
X_test_max = np.max(X_test)
sub_df = CreateSubmissionCSV(prediction / X_test_max)

(60980, 28)


In [0]:
sub_df

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,
HOBBIES_1_001_CA_1_validation,0,0,0,2,0,3,5,0,0,1,1,0,2,1,2,2,1,0,2,4,0,0,0,0,3,3,0,1
HOBBIES_1_002_CA_1_validation,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,2,1,1,0,0,0,0,0
HOBBIES_1_003_CA_1_validation,0,0,1,1,0,2,1,0,0,0,0,2,1,3,0,0,1,0,1,0,2,0,0,0,2,3,0,1
HOBBIES_1_004_CA_1_validation,0,0,1,2,4,1,6,4,0,0,0,2,2,4,2,1,1,1,1,1,0,4,0,1,3,0,2,6
HOBBIES_1_005_CA_1_validation,1,0,2,3,1,0,3,2,3,1,1,3,2,3,2,2,2,2,0,0,0,2,1,0,0,2,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FOODS_3_823_WI_3_evaluation,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,4,0
FOODS_3_824_WI_3_evaluation,0,0,0,0,0,0,0,0,0,2,4,7,9,11,13,15,13,0,6,0,20,22,22,22,22,22,23,23
FOODS_3_825_WI_3_evaluation,0,1,1,0,0,0,0,0,0,0,0,0,0,1,2,4,4,4,4,4,4,4,4,4,4,4,4,4


# クロスバリデーションのためのテスト

In [0]:
# データ読み出し
X_train = np.load(INPUT_DIR + "/TrainingTensor.npy")

In [0]:
from sklearn.model_selection import KFold

kfold = KFold(n_splits=5)
CV_gen = kfold.split(X_train[:1000])

In [29]:
next(CV_gen)[0].shape

(800,)

In [32]:
from keras.callbacks import EarlyStopping 

History = []

# 1000サンプルでクロスバリデーションテスト
for train_cv_idx, valid_cv_idx in kfold.split(X_train[:1000]):
    X_CV_train_gen = ReccurentTrainGenerator(X_train[train_cv_idx], batch_size=128)
    X_CV_valid_gen = ReccurentTrainGenerator(X_train[valid_cv_idx], batch_size=128)

    model = build_model() #カテゴリごとのモデルを作る時も、同様にfor文内で再度モデルをビルドすればよいかもしれない。
 
    # Early-stopping: patienceはもう少し大きくとる？
    early_stopping = EarlyStopping(patience=5, verbose=1) 

    history = model.fit_generator(X_CV_train_gen, epochs=500, verbose=1, validation_data=X_CV_valid_gen, callbacks=[early_stopping])
    History.append(history)

Epoch 1/500
16/16 [==============================] - 4s 267ms/step - loss: 0.8040 - accuracy: 0.5544 - val_loss: 10.0985 - val_accuracy: 0.5779
Epoch 2/500
16/16 [==============================] - 3s 169ms/step - loss: 0.5521 - accuracy: 0.5825 - val_loss: 1.1749 - val_accuracy: 0.6210
Epoch 3/500
16/16 [==============================] - 3s 169ms/step - loss: 0.4165 - accuracy: 0.6120 - val_loss: 0.3679 - val_accuracy: 0.6615
Epoch 4/500
16/16 [==============================] - 3s 170ms/step - loss: 0.3896 - accuracy: 0.6306 - val_loss: 0.3818 - val_accuracy: 0.6478
Epoch 5/500
16/16 [==============================] - 3s 170ms/step - loss: 0.3621 - accuracy: 0.6375 - val_loss: 0.3018 - val_accuracy: 0.6588
Epoch 6/500
16/16 [==============================] - 3s 172ms/step - loss: 0.3832 - accuracy: 0.6403 - val_loss: 0.3224 - val_accuracy: 0.6685
Epoch 7/500
16/16 [==============================] - 3s 168ms/step - loss: 0.3340 - accuracy: 0.6336 - val_loss: 0.3281 - val_accuracy: 0.670

In [33]:
History

# Loss を自作する(RMSSEをLossとして学習する)